## ఎంబెడ్డింగ్స్

ముందటి ఉదాహరణలో, మేము `vocab_size` పొడవు గల ఉన్నత-మానసిక బాగ్-ఆఫ్-వర్డ్స్ వెక్టార్లపై ఆపరేట్ చేసాము, మరియు తక్కువ-మానసిక స్థానిక ప్రాతినిధ్య వెక్టార్లను స్పార్స్ వన్-హాట్ ప్రాతినిధ్యంగా స్పష్టంగా మార్చాము. ఈ వన్-హాట్ ప్రాతినిధ్యం మెమరీ-సమర్థవంతంగా లేదు. అదనంగా, ప్రతి పదం ఒకదానితో మరొకటి స్వతంత్రంగా పరిగణించబడుతుంది, కాబట్టి వన్-హాట్ ఎన్‌కోడ్ చేసిన వెక్టార్లు పదాల మధ్య అర్థ సంబంధాలను వ్యక్తం చేయవు.

ఈ యూనిట్‌లో, మేము **News AG** డేటాసెట్‌ను కొనసాగించి పరిశీలిస్తాము. మొదలుపెట్టడానికి, డేటాను లోడ్ చేసి, గత యూనిట్ నుండి కొన్ని నిర్వచనాలను పొందుదాం.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### ఎంబెడ్డింగ్ అంటే ఏమిటి?

**ఎంబెడ్డింగ్** అనేది పదాలను తక్కువ-మితి గాఢ వెక్టర్లుగా ప్రాతినిధ్యం వహించడం, అవి ఆ పదం యొక్క అర్థాన్ని ప్రతిబింబిస్తాయి. మనం తర్వాత అర్థవంతమైన పద ఎంబెడ్డింగ్స్ ఎలా నిర్మించాలో చర్చిస్తాము, కానీ ఇప్పటి కోసం ఎంబెడ్డింగ్స్ అనేవి పద వెక్టర్ యొక్క మితిని తగ్గించే ఒక విధానం అని భావిద్దాం.

కాబట్టి, ఒక ఎంబెడ్డింగ్ లేయర్ ఒక పదాన్ని ఇన్‌పుట్‌గా తీసుకుని, నిర్దిష్ట `embedding_size` ఉన్న అవుట్‌పుట్ వెక్టర్‌ను ఉత్పత్తి చేస్తుంది. ఒక అర్థంలో, ఇది `Dense` లేయర్‌కు చాలా సమానంగా ఉంటుంది, కానీ ఒక-హాట్ ఎన్‌కోడ్ చేసిన వెక్టర్‌ను ఇన్‌పుట్‌గా తీసుకోవడం కాకుండా, ఇది పద సంఖ్యను తీసుకోవచ్చు.

మన నెట్‌వర్క్‌లో మొదటి లేయర్‌గా ఎంబెడ్డింగ్ లేయర్‌ను ఉపయోగించడం ద్వారా, మనం బాగ్-ఆఫ్-వర్డ్స్ నుండి **ఎంబెడ్డింగ్ బాగ్** మోడల్‌కు మారవచ్చు, ఇక్కడ మనం మొదట మన టెక్స్ట్‌లోని ప్రతి పదాన్ని సంబంధిత ఎంబెడ్డింగ్‌గా మార్చి, ఆ ఎంబెడ్డింగ్స్ మొత్తం మీద `sum`, `average` లేదా `max` వంటి ఏదైనా సమాహార ఫంక్షన్‌ను లెక్కిస్తాము.

![ఐదు వరుస పదాల కోసం ఎంబెడ్డింగ్ క్లాసిఫయర్ చూపిస్తున్న చిత్రం.](../../../../../translated_images/te/embedding-classifier-example.b77f021a7ee67eee.webp)

మన క్లాసిఫయర్ న్యూరల్ నెట్‌వర్క్ క్రింది లేయర్లతో కూడి ఉంటుంది:

* `TextVectorization` లేయర్, ఇది ఒక స్ట్రింగ్‌ను ఇన్‌పుట్‌గా తీసుకుని, టోకెన్ సంఖ్యల టెన్సర్‌ను ఉత్పత్తి చేస్తుంది. మనం కొంత తగిన వోకాబ్యులరీ పరిమాణం `vocab_size` ను నిర్దేశించి, తక్కువగా ఉపయోగించే పదాలను పక్కన పెడతాము. ఇన్‌పుట్ ఆకారం 1 ఉంటుంది, అవుట్‌పుట్ ఆకారం $n$ ఉంటుంది, ఎందుకంటే మనం $n$ టోకెన్లు పొందుతాము, వాటిలో ప్రతి ఒక్కటి 0 నుండి `vocab_size` వరకు సంఖ్యలను కలిగి ఉంటుంది.
* `Embedding` లేయర్, ఇది $n$ సంఖ్యలను తీసుకుని, ప్రతి సంఖ్యను ఒక నిర్దిష్ట పొడవు (మన ఉదాహరణలో 100) గాఢ వెక్టర్‌గా తగ్గిస్తుంది. కాబట్టి, $n$ ఆకారపు ఇన్‌పుట్ టెన్సర్ $n\times 100$ ఆకారపు టెన్సర్‌గా మారుతుంది.
* సమాహార లేయర్, ఇది ఈ టెన్సర్‌ను మొదటి అక్షం మీద సగటు తీసుకుంటుంది, అంటే వేర్వేరు పదాలకు సంబంధించిన అన్ని $n$ ఇన్‌పుట్ టెన్సర్ల సగటును లెక్కిస్తుంది. ఈ లేయర్‌ను అమలు చేయడానికి, మనం `Lambda` లేయర్‌ను ఉపయోగించి, సగటు లెక్కించే ఫంక్షన్‌ను అందిస్తాము. అవుట్‌పుట్ ఆకారం 100 ఉంటుంది, ఇది మొత్తం ఇన్‌పుట్ వరుస యొక్క సంఖ్యాత్మక ప్రాతినిధ్యం.
* చివరి `Dense` లీనియర్ క్లాసిఫయర్.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` ప్రింట్ అవుట్‌లో, **output shape** కాలమ్‌లో, మొదటి టెన్సర్ డైమెన్షన్ `None` మినీబ్యాచ్ సైజ్‌కు సంబంధించినది, మరియు రెండవది టోకెన్ సీక్వెన్స్ పొడవుకు సంబంధించినది. మినీబ్యాచ్‌లోని అన్ని టోకెన్ సీక్వెన్సులు వేర్వేరు పొడవులు కలిగి ఉంటాయి. దీన్ని ఎలా నిర్వహించాలో తదుపరి విభాగంలో చర్చిస్తాము.

ఇప్పుడు నెట్‌వర్క్‌ను ట్రెయిన్ చేద్దాం:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **గమనిక** మేము డేటా యొక్క ఒక ఉపసమూహం ఆధారంగా వెక్టరైజర్‌ను నిర్మిస్తున్నాము. ఇది ప్రక్రియను వేగవంతం చేయడానికి చేయబడింది, మరియు ఇది మా టెక్స్ట్ నుండి అన్ని టోకెన్లు శబ్దకోశంలో ఉండకపోవచ్చు అనే పరిస్థితికి దారితీస్తుంది. ఈ సందర్భంలో, ఆ టోకెన్లు నిర్లక్ష్యం చేయబడతాయి, ఇది కొంతమేర తక్కువ ఖచ్చితత్వానికి కారణమవుతుంది. అయితే, వాస్తవ జీవితంలో టెక్స్ట్ యొక్క ఉపసమూహం తరచుగా మంచి శబ్దకోశ అంచనాను ఇస్తుంది.


### వేరియబుల్ సీక్వెన్స్ పరిమాణాలతో వ్యవహరించడం

మినీబ్యాచ్లలో శిక్షణ ఎలా జరుగుతుందో మనం అర్థం చేసుకుందాం. పై ఉదాహరణలో, ఇన్‌పుట్ టెన్సర్ పరిమాణం 1 ఉంది, మరియు మనం 128-లంబమైన మినీబ్యాచ్లను ఉపయోగిస్తాము, కాబట్టి టెన్సర్ యొక్క వాస్తవ పరిమాణం $128 \times 1$ అవుతుంది. అయితే, ప్రతి వాక్యంలో ఉన్న టోకెన్ల సంఖ్య భిన్నంగా ఉంటుంది. ఒకే ఇన్‌పుట్‌కు `TextVectorization` లేయర్‌ను వర్తింపజేస్తే, టోకెన్ల సంఖ్య టెక్స్ట్ ఎలా టోకెనైజ్ చేయబడిందో ఆధారపడి వేరుగా ఉంటుంది:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


అయితే, మనం వెక్టరైజర్‌ను అనేక సీక్వెన్సులకు వర్తింపజేసినప్పుడు, అది ఒక చతురస్రాకార టెన్సర్‌ను ఉత్పత్తి చేయాలి, కాబట్టి ఉపయోగించని అంశాలను PAD టోకెన్‌తో (మన సందర్భంలో ఇది సున్నా) నింపుతుంది:


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

ఇక్కడ మనం ఎంబెడ్డింగ్స్‌ను చూడవచ్చు:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **గమనిక**: ప్యాడింగ్ పరిమాణాన్ని తగ్గించడానికి, కొన్ని సందర్భాల్లో డేటాసెట్‌లోని అన్ని సీక్వెన్సులను పెరుగుతున్న పొడవు (లేదా, మరింత ఖచ్చితంగా, టోకెన్ల సంఖ్య) క్రమంలో సర్దుబాటు చేయడం మంచిది. ఇది ప్రతి మినీబ్యాచ్‌లో సమానమైన పొడవు ఉన్న సీక్వెన్సులు ఉండేలా చేస్తుంది.


## సేమాంటిక్ ఎంబెడ్డింగ్స్: Word2Vec

మా గత ఉదాహరణలో, ఎంబెడ్డింగ్ లేయర్ పదాలను వెక్టర్ ప్రాతినిధ్యాలకు మ్యాప్ చేయడం నేర్చుకుంది, అయితే ఆ ప్రాతినిధ్యాలకు సేమాంటిక్ అర్థం ఉండలేదు. సమానమైన పదాలు లేదా సమానార్థక పదాలు ఒకదానికి దగ్గరగా ఉండే వెక్టర్లుగా ఉండేలా వెక్టర్ ప్రాతినిధ్యం నేర్చుకోవడం బాగుంటుంది (ఉదాహరణకు యూక్లిడియన్ దూరం ఆధారంగా).

అందుకోసం, పెద్ద పరిమాణంలో ఉన్న టెక్స్ట్ సేకరణపై [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) వంటి సాంకేతికత ఉపయోగించి మా ఎంబెడ్డింగ్ మోడల్‌ను ప్రీట్రెయిన్ చేయాలి. ఇది రెండు ప్రధాన ఆర్కిటెక్చర్లపై ఆధారపడి ఉంటుంది, ఇవి పదాల పంపిణీ ప్రాతినిధ్యాన్ని ఉత్పత్తి చేయడానికి ఉపయోగిస్తారు:

 - **కంటిన్యూయస్ బ్యాగ్-ఆఫ్-వర్డ్స్** (CBoW), ఇందులో మోడల్ చుట్టుపక్కల ఉన్న సందర్భం నుండి ఒక పదాన్ని అంచనా వేయడానికి శిక్షణ పొందుతుంది. ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ ఇచ్చినప్పుడు, మోడల్ లక్ష్యం $(W_{-2},W_{-1},W_1,W_2)$ నుండి $W_0$ ను అంచనా వేయడం.
 - **కంటిన్యూయస్ స్కిప్-గ్రామ్** CBoWకి వ్యతిరేకంగా ఉంటుంది. మోడల్ ప్రస్తుత పదాన్ని అంచనా వేయడానికి చుట్టుపక్కల ఉన్న సందర్భ పదాల విండోను ఉపయోగిస్తుంది.

CBoW వేగంగా ఉంటుంది, స్కిప్-గ్రామ్ మందగించగా, అరుదైన పదాలను బాగా ప్రాతినిధ్యం చేయగలదు.

![పదాలను వెక్టర్లుగా మార్చడానికి CBoW మరియు స్కిప్-గ్రామ్ అల్గోరిథమ్స్ రెండింటినీ చూపించే చిత్రం.](../../../../../translated_images/te/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News డేటాసెట్‌పై ప్రీట్రెయిన్ చేసిన Word2Vec ఎంబెడ్డింగ్‌తో ప్రయోగం చేయడానికి, **gensim** లైబ్రరీని ఉపయోగించవచ్చు. క్రింద 'neural' కు అత్యంత సమానమైన పదాలను కనుగొంటాం.

> **Note:** మీరు మొదటిసారి word vectors సృష్టించినప్పుడు, వాటిని డౌన్లోడ్ చేయడానికి కొంత సమయం పట్టవచ్చు!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


మేము శబ్దం నుండి వెక్టర్ ఎంబెడ్డింగ్‌ను కూడా తీసుకోవచ్చు, ఇది వర్గీకరణ మోడల్ శిక్షణలో ఉపయోగించబడుతుంది. ఎంబెడ్డింగ్‌లో 300 భాగాలు ఉంటాయి, కానీ స్పష్టత కోసం ఇక్కడ మేము వెక్టర్ యొక్క మొదటి 20 భాగాలను మాత్రమే చూపిస్తున్నాము:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

సెమాంటిక్ ఎంబెడ్డింగ్స్ గురించి గొప్ప విషయం ఏమిటంటే, మీరు సెమాంటిక్స్ ఆధారంగా వెక్టర్ ఎంకోడింగ్‌ను మానిప్యులేట్ చేయవచ్చు. ఉదాహరణకు, మనం వెక్టర్ ప్రాతినిధ్యం *king* మరియు *woman* అనే పదాలకు όσο దగ్గరగా ఉండే, మరియు *man* అనే పదం నుండి όσο దూరంగా ఉండే పదాన్ని కనుగొనమని అడగవచ్చు:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

పై ఉదాహరణలో కొంత అంతర్గత GenSym మాయాజాలం ఉపయోగించబడింది, కానీ ప్రాథమిక తర్కం నిజంగా చాలా సులభం. ఎంబెడ్డింగ్స్ గురించి ఒక ఆసక్తికరమైన విషయం ఏమిటంటే, మీరు ఎంబెడ్డింగ్ వెక్టర్లపై సాధారణ వెక్టర్ ఆపరేషన్లు చేయవచ్చు, మరియు అది పదాల **అర్థాలు** పై ఆపరేషన్లను ప్రతిబింబిస్తుంది. పై ఉదాహరణను వెక్టర్ ఆపరేషన్ల రూపంలో వ్యక్తపరచవచ్చు: మేము **KING-MAN+WOMAN** కు సరిపోలే వెక్టర్‌ను లెక్కిస్తాము (ఆపరేషన్లు `+` మరియు `-` సంబంధిత పదాల వెక్టర్ ప్రాతినిధ్యాలపై నిర్వహించబడతాయి), ఆ తర్వాత ఆ వెక్టర్‌కు నిఘంటువు లో అత్యంత సమీపమైన పదాన్ని కనుగొంటాము:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: మేము *man* మరియు *woman* వెక్టార్లకు చిన్న కోఎఫిషియెంట్లు జోడించాల్సి వచ్చింది - వాటిని తీసివేసి చూడండి, ఏమి జరుగుతుందో తెలుసుకోండి.

సమీపమైన వెక్టర్‌ను కనుగొనడానికి, మన వెక్టర్ మరియు శబ్దకోశంలోని అన్ని వెక్టార్ల మధ్య దూరాల వెక్టర్‌ను లెక్కించడానికి TensorFlow యంత్రాంగాన్ని ఉపయోగిస్తాము, ఆపై `argmin` ఉపయోగించి కనిష్టమైన పదం యొక్క సూచికను కనుగొంటాము.


Word2Vec పదార్థాల సారాంశాన్ని వ్యక్తపరచడానికి మంచి మార్గంగా కనిపించినప్పటికీ, దీనికి అనేక లోపాలు ఉన్నాయి, వాటిలో ముఖ్యమైనవి:

* CBoW మరియు skip-gram మోడల్స్ రెండూ **అనుమానాత్మక ఎంబెడ్డింగ్స్** (predictive embeddings) కాగా, అవి కేవలం స్థానిక సందర్భాన్ని మాత్రమే పరిగణలోకి తీసుకుంటాయి. Word2Vec గ్లోబల్ సందర్భాన్ని ఉపయోగించదు.
* Word2Vec పద **మార్ఫాలజీ** (morphology), అంటే పదార్థం పదంలోని వేరే భాగాలపై ఆధారపడి ఉండటం, ఉదాహరణకు మూలం (root) వంటి అంశాలను పరిగణలోకి తీసుకోదు.

**FastText** రెండవ పరిమితిని అధిగమించడానికి ప్రయత్నిస్తుంది, మరియు Word2Vec పై ఆధారపడి ప్రతి పదం మరియు ఆ పదంలో ఉన్న అక్షర n-గ్రామ్లకు వెక్టర్ ప్రాతినిధ్యాలను నేర్చుకుంటుంది. ఈ ప్రాతినిధ్యాల విలువలను ప్రతి శిక్షణ దశలో ఒక వెక్టర్‌గా సగటు చేస్తుంది. ఇది ప్రీట్రైనింగ్‌కు ఎక్కువ గణనను జోడించినప్పటికీ, పద ఎంబెడ్డింగ్స్‌కు ఉప-పద సమాచారాన్ని సంకేతీకరించడానికి వీలు కల్పిస్తుంది.

మరొక విధానం, **GloVe**, పద ఎంబెడ్డింగ్స్‌కు వేరే దృక్పథాన్ని ఉపయోగిస్తుంది, ఇది పద-సందర్భ మేట్రిక్స్ యొక్క ఫ్యాక్టరైజేషన్‌పై ఆధారపడి ఉంటుంది. మొదట, ఇది వివిధ సందర్భాలలో పదాల సంభవాల సంఖ్యను లెక్కించే పెద్ద మేట్రిక్స్‌ను నిర్మిస్తుంది, తరువాత ఈ మేట్రిక్స్‌ను తక్కువ కొలతలలో ప్రతినిధ్యం చేయడానికి ప్రయత్నిస్తుంది, పునర్నిర్మాణ నష్టాన్ని తగ్గించే విధంగా.

gensim లైబ్రరీ ఆ పద ఎంబెడ్డింగ్స్‌ను మద్దతు ఇస్తుంది, మరియు మీరు పై మోడల్ లోడింగ్ కోడ్‌ను మార్చి వాటితో ప్రయోగాలు చేయవచ్చు.


## Kerasలో pretrained embeddings ఉపయోగించడం

ముందుగా ఇచ్చిన ఉదాహరణను మేము మార్చి, embedding లేయర్‌లోని మ్యాట్రిక్స్‌ను Word2Vec వంటి సేమాంటిక్ embeddings తో ముందుగా నింపవచ్చు. pretrained embedding మరియు టెక్స్ట్ కార్పస్ యొక్క పదసంపదలు సాధారణంగా సరిపోలకపోవచ్చు, కాబట్టి మేము ఒకదాన్ని ఎంచుకోవాలి. ఇక్కడ మేము రెండు సాధ్యమైన ఎంపికలను పరిశీలిస్తాము: tokenizer పదసంపదను ఉపయోగించడం, మరియు Word2Vec embeddings నుండి పదసంపదను ఉపయోగించడం.

### tokenizer పదసంపదను ఉపయోగించడం

tokenizer పదసంపదను ఉపయోగించినప్పుడు, పదసంపదలోని కొన్ని పదాలకు Word2Vec embeddings ఉంటాయి, మరికొన్ని ఉండకపోవచ్చు. మన పదసంపద పరిమాణం `vocab_size` అని, Word2Vec embedding వెక్టర్ పొడవు `embed_size` అని తీసుకుంటే, embedding లేయర్ ఒక `vocab_size`$\times$`embed_size` ఆకారంలో ఉన్న వెయిట్ మ్యాట్రిక్స్ ద్వారా ప్రాతినిధ్యం వహిస్తుంది. మేము ఈ మ్యాట్రిక్స్‌ను పదసంపదలోని ప్రతి పదం ద్వారా వెళ్ళి నింపుతాము:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Word2Vec శబ్దకోశంలో లేని పదాలకు, వాటిని సున్నాలుగా ఉంచవచ్చు లేదా యాదృచ్ఛిక వెక్టర్‌ను సృష్టించవచ్చు.

ఇప్పుడు మేము ప్రీట్రెయిన్డ్ వెయిట్స్‌తో ఒక ఎంబెడ్డింగ్ లేయర్‌ను నిర్వచించవచ్చు:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

ఇప్పుడు మన మోడల్‌ను శిక్షణ ఇవ్వండి.


In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **గమనిక**: `Embedding` సృష్టించినప్పుడు `trainable=False` గా సెట్ చేసినట్లు గమనించండి, అంటే మనం Embedding లేయర్‌ను మళ్లీ శిక్షణ ఇవ్వడం లేదు. దీని వల్ల ఖచ్చితత్వం కొంచెం తక్కువగా ఉండవచ్చు, కానీ శిక్షణ వేగవంతం అవుతుంది.

### embedding పదసంపద ఉపయోగించడం

మునుపటి విధానంలో ఒక సమస్య ఏమిటంటే TextVectorization మరియు Embedding లో ఉపయోగించిన పదసంపదలు వేరుగా ఉంటాయి. ఈ సమస్యను అధిగమించడానికి, మనం క్రింది పరిష్కారాల్లో ఒకదాన్ని ఉపయోగించవచ్చు:
* మన పదసంపదపై Word2Vec మోడల్‌ను మళ్లీ శిక్షణ ఇవ్వడం.
* ప్రీట్రెయిన్డ్ Word2Vec మోడల్ నుండి వచ్చిన పదసంపదతో మన డేటాసెట్‌ను లోడ్ చేయడం. డేటాసెట్ లోడ్ చేసే సమయంలో ఉపయోగించే పదసంపదను పేర్కొనవచ్చు.

రెండవ విధానం సులభంగా కనిపిస్తుందని, అందువల్ల దాన్ని అమలు చేద్దాం. మొదటగా, Word2Vec embeddings నుండి తీసుకున్న నిర్దిష్ట పదసంపదతో `TextVectorization` లేయర్‌ను సృష్టిస్తాం:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

gensim పదం ఎంబెడ్డింగ్స్ లైబ్రరీలో ఒక సౌకర్యవంతమైన ఫంక్షన్ ఉంది, `get_keras_embeddings`, ఇది మీ కోసం తగిన Keras ఎంబెడ్డింగ్స్ లేయర్‌ను ఆటోమేటిక్‌గా సృష్టిస్తుంది.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


మేము ఎక్కువ ఖచ్చితత్వం చూడకపోవడానికి ఒక కారణం మా డేటాసెట్ నుండి కొన్ని పదాలు ప్రీట్రెయిన్డ్ GloVe పదసంపదలో లేవు, అందువల్ల అవి ప్రాథమికంగా నిర్లక్ష్యం చేయబడతాయి. దీన్ని అధిగమించడానికి, మేము మా డేటాసెట్ ఆధారంగా మా స్వంత ఎంబెడ్డింగ్స్‌ను శిక్షణ ఇవ్వవచ్చు.


## సందర్భానుసారమైన ఎంబెడ్డింగ్స్

Word2Vec వంటి సాంప్రదాయ ప్రీట్రెయిన్డ్ ఎంబెడ్డింగ్ ప్రాతినిధ్యాల ఒక ముఖ్యమైన పరిమితి ఏమిటంటే, అవి ఒక పదం యొక్క కొంత అర్థాన్ని పట్టుకోవచ్చు గానీ, వేర్వేరు అర్థాలను వేరుచేయలేవు. ఇది తరువాతి మోడల్స్‌లో సమస్యలు కలిగించవచ్చు.

ఉదాహరణకు, 'play' అనే పదానికి ఈ రెండు వాక్యాలలో వేర్వేరు అర్థాలు ఉన్నాయి:
- నేను థియేటర్‌లో ఒక **play** కి వెళ్లాను.
- జాన్ తన స్నేహితులతో **play** చేయాలనుకుంటున్నాడు.

మనం చర్చించిన ప్రీట్రెయిన్డ్ ఎంబెడ్డింగ్స్ 'play' పదం యొక్క రెండు అర్థాలను ఒకే ఎంబెడ్డింగ్‌లో ప్రాతినిధ్యం చేస్తాయి. ఈ పరిమితిని అధిగమించడానికి, పెద్ద టెక్స్ట్ కార్పస్‌పై శిక్షణ పొందిన **భాషా మోడల్** ఆధారంగా ఎంబెడ్డింగ్స్ నిర్మించాలి, ఇది పదాలు వేర్వేరు సందర్భాలలో ఎలా కలిపి ఉండవచ్చో *తెలుసుకుంటుంది*. సందర్భానుసారమైన ఎంబెడ్డింగ్స్ గురించి చర్చించడం ఈ ట్యుటోరియల్ పరిధికి బయట, కానీ భాషా మోడల్స్ గురించి తదుపరి యూనిట్‌లో మళ్లీ వాటిని చూడబోతున్నాం.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలోనే అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం చేయించుకోవడం మంచిది. ఈ అనువాదం వలన కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారుల బాధ్యత మేము తీసుకోము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
